## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_2 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o2/o2_bincore6.csv", index_col=0)
print(bin6_2.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_2.reset_index(inplace=True, drop=True)

print('shape -', bin6_2.shape)
print('reset_index 완료')
print('input data shape')
bin6_2.head()

(28456104, 2)
shape - (28456104, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_2['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_2['label'].value_counts())

256
0    28418383
1       37721
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_2[bin6_2['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 96 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_2), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_2', len(ls_idx_bin))

# loc 로 수정필요
bin6_2_Ngram = bin6_2.loc[ls_idx_bin,:].copy()

3621216
나머지 0
최종 길이 3621216
bin6_2 3621216


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_2_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_2.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_2)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_2[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

37721.0
0 5261660
1000 1125660
2000 26486793
3000 17001166
4000 17055757
5000 3777717
6000 19913397
7000 24026294
8000 20299685
9000 8396486
10000 19826163
11000 7588681
12000 7635670
13000 10324418
14000 22537693
15000 10851466
16000 2542797
17000 18950923
18000 13593564
19000 7680419
20000 8899177
21000 2354571
22000 28180068
23000 16603112
24000 21325770
25000 16737954
26000 7497857
27000 799075
28000 8768765
29000 7640022
30000 18230076
31000 21597336
32000 27576946
33000 25026605
34000 12824720
35000 23487737
36000 23432437
37000 24835101
완료
37721


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_2['label'].value_counts()[1])

37721
37721


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_2_Ngram])
final.shape

(7242432, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_2_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_2_onehot_Ngram = pd.concat([final['label'], bc6_2_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_2_onehot_Ngram.shape)

원핫인코딩완료
(7242432, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_2 = bc6_2_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_2 = bc6_2_onehot_Ngram['label'].to_numpy()
print(x_bc6_2.shape, x_bc6_2.shape)

x_bc6_2 = x_bc6_2.reshape(-1, right_idx, x_bc6_2.shape[1])
y_bc6_2 = y_bc6_2.reshape(-1, right_idx, 1)

print(x_bc6_2.shape, y_bc6_2.shape)

(7242432, 256) (7242432, 256)
(75442, 96, 256) (75442, 96, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_2.shape[0])

x_bc6_2 = x_bc6_2[p]
y_bc6_2 = y_bc6_2[p]

print(x_bc6_2.shape, y_bc6_2.shape)

(75442, 96, 256) (75442, 96, 1)


In [10]:
np.save('x_bc6_2', x_bc6_2)
np.save('y_bc6_2', y_bc6_2)

## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(144, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [11]:
model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bc6_2,
           y_bc6_2,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()

print('걸린시간:', endtime - starttime)
model1.save('gcc6_bin_core_s96_h144_o2.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 96, 288)           461952    
_________________________________________________________________
time_distributed (TimeDistri (None, 96, 1)             289       
Total params: 462,241
Trainable params: 462,241
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 52809 samples, validate on 22633 samples
Epoch 1/500
52809/52809 [==============================] - 49s 919us/sample - loss: 0.0090 - accuracy: 0.9972 - val_loss: 0.0018 - val_accuracy: 0.9994
Epoch 2/500
52809/52809 [==============================] - 35s 655us/sample - loss: 0.0013 - accuracy: 0.9996 - val_loss: 9.9970e-04 - val_accuracy: 0.9997


걸린시간: 769.3008892536163
save 완료
